In [1]:
import time
import tensorflow as tf

d:\Users\User\anaconda3\envs\tf_gpu_2_6_0_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Users\User\anaconda3\envs\tf_gpu_2_6_0_env\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
d:\Users\User\anaconda3\envs\tf_gpu_2_6_0_env\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from object_detection.utils import label_map_util

ANNOTATION_PATH = 'data/test/creatures_label_map.pbtxt' 

category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH, use_display_name=True)

print(category_index)

{1: {'id': 1, 'name': 'fish'}, 2: {'id': 2, 'name': 'jellyfish'}, 3: {'id': 3, 'name': 'penguin'}, 4: {'id': 4, 'name': 'puffin'}, 5: {'id': 5, 'name': 'shark'}, 6: {'id': 6, 'name': 'starfish'}, 7: {'id': 7, 'name': 'stingray'}}


In [3]:
PATH_TO_SAVED_MODEL = "./workspace/models/aquarium_model/exported_models/saved_model"


print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detector = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 11.488907814025879 seconds


In [7]:
import cv2
from object_detection.utils import visualization_utils as viz_utils
import numpy as np

def apply_detector_to_video(video_input_path, video_output_path, detector, category_index, min_score_thresh=0.3):
    # Open the video file
    video_reader = cv2.VideoCapture(video_input_path)

    # Get the width, height, and frames per second (fps) of the video
    width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # fps = video_reader.get(cv2.CAP_PROP_FPS)
    fps = 32.75

    # Create a VideoWriter object to write the output video
    video_writer = cv2.VideoWriter(video_output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    while video_reader.isOpened():
        ret, frame = video_reader.read()
        if not ret:
            break

        # Convert the image to a tensor
        input_tensor = tf.convert_to_tensor(frame)
        input_tensor = input_tensor[tf.newaxis, ...]

        # Apply the model to the frame
        detections = detector(input_tensor)
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        detections['num_detections'] = num_detections
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        # Visualize the predicted bboxes on the frame
        viz_utils.visualize_boxes_and_labels_on_image_array(
                frame,
                detections['detection_boxes'],
                detections['detection_classes'],
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=200,
                min_score_thresh=min_score_thresh,
                agnostic_mode=False)

        # Write the frame to the output video file
        video_writer.write(frame)

    # Close the video file
    video_reader.release()
    video_writer.release()

# Now you can call apply_detector_to_video with the paths to your input and output videos.
apply_detector_to_video('input1.mp4', 'output1.mp4', detector, category_index)
